In [15]:
%matplotlib inline
from __future__ import print_function
import os,sys
notebook_code_root = os.path.dirname(os.getcwd())
data_root = os.path.dirname(notebook_code_root) + '/data/'
print("Data root: %s" % data_root)
sys.path.insert(0,notebook_code_root)

import cutils
from cutils import *

from keras.layers.core import Reshape
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

from vgg16 import *

opts = NotebookData(data_dir=data_root, sample_mode=False, preprocess=True)
print(opts)

Data root: /home/ubuntu/data/
Options:
  Testing directory: /home/ubuntu/data/statefarm/test/
  Training directory: /home/ubuntu/data/statefarm/train/
  Validation directory: /home/ubuntu/data/statefarm/valid/
  Preprocess directory: /home/ubuntu/data/statefarm/preprocessed/
  Results directory: /home/ubuntu/data/statefarm/results


In [2]:
# Load the VGG model.
vgg = Vgg16()

In [3]:
ls = vgg.model.layers
idx = 0
for i in range(len(ls)):
    if type(ls[i]) is MaxPooling2D:
        idx = i

conv_model = Sequential(layers=ls[:(idx+1)])
fc_layers = ls[(idx+1):]

In [4]:
def get_data(opt):
    tf = None
    vf = None
    tc = None
    vc = None
    trn_dir = os.path.join(opt.data_root, 'preprocessed', 'conv')
    if opt.preprocess:
        gen = image.ImageDataGenerator()
        img_trn_batches = gen.flow_from_directory(opt.train_dir(),
                                                  target_size=(224,224),
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=64)
        img_val_batches = gen.flow_from_directory(opt.valid_dir(),
                                                  target_size=(224,224),
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=64)
        tc = to_categorical(img_trn_batches.classes)
        vc = to_categorical(img_val_batches.classes)
        tf = conv_model.predict_generator(img_trn_batches, verbose=1)
        vf = conv_model.predict_generator(img_val_batches, verbose=1)
        if not os.path.exists(trn_dir):
            os.makedirs(trn_dir)
        save_array(os.path.join(trn_dir, 'train-dt'), tf)
        save_array(os.path.join(trn_dir, 'val-dt'), vf)
        save_array(os.path.join(trn_dir, 'train-cl'), tc)
        save_array(os.path.join(trn_dir, 'val-cl'), vc)
    else:
        tf = load_array(os.path.join(trn_dir, 'train-dt'))
        vf = load_array(os.path.join(trn_dir, 'val-dt'))
        tc = load_array(os.path.join(trn_dir, 'train-cl'))
        vc = load_array(os.path.join(trn_dir, 'val-cl'))
        
    return (tf, vf, tc, vc)

In [5]:
trn_features, val_features, img_trn_classes, img_val_classes = get_data(opts)

Found 17622 images belonging to 10 classes.
Found 4802 images belonging to 10 classes.
76/76 [==============================] - 92s 1s/step


In [6]:
def rand_training(trn, img):
    zipped_trn = zip(trn, img)
    np.random.shuffle(zipped_trn)
    trn_data = np.stack([data for data, cl in zipped_trn], axis=0)
    trn_cls = np.stack([cl for data, cl in zipped_trn], axis=0)
    return trn_data, trn_cls

In [82]:
# Create a new model with only dense layers.
fc_model = Sequential([
     # Input shape is shape of the conv result.
     Reshape((25088,), input_shape=(512, 7, 7,)),
     Dense(64, activation="relu"),
     BatchNormalization(),
     Dropout(0.2),
     Dense(64, activation='relu'),
     BatchNormalization(),
     Dropout(0.3),
     Dense(64, activation='relu'),
     BatchNormalization(),
     Dropout(0.5),
     Dense(10, activation='softmax')
    ])

In [84]:
fc_model.compile(optimizer=Adam(lr=0.000001),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [85]:
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 4s 199us/step - loss: 3.3783 - acc: 0.1138 - val_loss: 2.6165 - val_acc: 0.1054
Epoch 2/2
17622/17622 [==============================] - 3s 192us/step - loss: 3.2218 - acc: 0.1281 - val_loss: 2.5958 - val_acc: 0.1120
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 3.0870 - acc: 0.1456 - val_loss: 2.5771 - val_acc: 0.1137
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 2.9813 - acc: 0.1593 - val_loss: 2.5734 - val_acc: 0.1187
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 2.8640 - acc: 0.1705 - val_loss: 2.5716 - val_acc: 0.1241
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 2.7497 - acc: 0.1901 - val_loss: 2.5609 - val_acc: 0.1295
Train on 17622 samples, validat

In [86]:
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 199us/step - loss: 0.5932 - acc: 0.8309 - val_loss: 1.6556 - val_acc: 0.5192
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.5884 - acc: 0.8275 - val_loss: 1.6429 - val_acc: 0.5246
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.5786 - acc: 0.8312 - val_loss: 1.6317 - val_acc: 0.5281
Epoch 2/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.5761 - acc: 0.8376 - val_loss: 1.6303 - val_acc: 0.5271
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.5566 - acc: 0.8456 - val_loss: 1.6423 - val_acc: 0.5233
Epoch 2/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.5554 - acc: 0.8448 - val_loss: 1.6348 - val_acc: 0.5264
Train on 17622 samples, validat

In [87]:
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 4s 200us/step - loss: 0.2135 - acc: 0.9598 - val_loss: 1.4181 - val_acc: 0.6029
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.2054 - acc: 0.9620 - val_loss: 1.4165 - val_acc: 0.6027
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.2155 - acc: 0.9594 - val_loss: 1.4115 - val_acc: 0.6052
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.2041 - acc: 0.9633 - val_loss: 1.4129 - val_acc: 0.6050
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.2068 - acc: 0.9603 - val_loss: 1.4173 - val_acc: 0.6010
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.2048 - acc: 0.9646 - val_loss: 1.4207 - val_acc: 0.6035
Train on 17622 samples, validat

In [88]:
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 4s 200us/step - loss: 0.0992 - acc: 0.9878 - val_loss: 1.3589 - val_acc: 0.6266
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.1003 - acc: 0.9864 - val_loss: 1.3573 - val_acc: 0.6302
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.0987 - acc: 0.9869 - val_loss: 1.3704 - val_acc: 0.6264
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.0975 - acc: 0.9868 - val_loss: 1.3618 - val_acc: 0.6297
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.0981 - acc: 0.9858 - val_loss: 1.3815 - val_acc: 0.6235
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.0948 - acc: 0.9874 - val_loss: 1.3766 - val_acc: 0.6233
Train on 17622 samples, validat

In [89]:
process_model(fc_model, opts, 'v1', sub=True)

Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch0
10000/10000 [==============================] - 1s 122us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch1
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch2
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch3
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch4
10000/10000 [==============================] - 1s 109us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch5
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch6
10000/10000 [==============================] - 1s 109us/step
Predicting batch at /home/ubuntu/data/statefarm/preproc

In [11]:
from IPython.display import FileLink

FileLink('submission.csv')

/home/ubuntu/nbs/lesson-3/submission.csv